# Установка библиотек и импорты

Удаляем albumentations и ultralytics для предотвращения ошибки.

In [ ]:
!pip uninstall ultralytics
!pip uninstall albumentations -y

Удаляем папки, созданные в предыдущих запусках

In [ ]:
!rm -rf "/kaggle/working/DocLayout YOLO"
!rm -rf /kaggle/working/wandb
!rm -rf /kaggle/working/config

In [ ]:
!rm -rf "/kaggle/working/datasets"

Скачиваем библиотеки для работы DocLayout-YOLO

In [ ]:
!pip install doclayout_yolo
!pip install huggingface_hub

Пишем стандартные импорты

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from pathlib import Path
from doclayout_yolo import YOLOv10
from huggingface_hub import hf_hub_download
import json
import shutil
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wandb --q

import wandb

secret_value = 'мой ключ от wandb'

wandb.login(key=secret_value)

# Подготовка датасета

Переводим датасет в YOLO формат

In [ ]:
import os
import shutil
import json
import yaml
import random
from pathlib import Path

# пути к данным
SOURCE_IMAGES_DIR = '/kaggle/input/final-dataset/raw_data/images'
SOURCE_JSONS_DIR = '/kaggle/input/final-dataset/raw_data/jsons'
OUTPUT_BASE_DIR = '/kaggle/working/custom_data'

# очищаем выходные директории если они существуют
if os.path.exists(OUTPUT_BASE_DIR):
    shutil.rmtree(OUTPUT_BASE_DIR)

# создаем необходимые директории
for dir_path in [
    f'{OUTPUT_BASE_DIR}/images/train',
    f'{OUTPUT_BASE_DIR}/images/val',
    f'{OUTPUT_BASE_DIR}/labels/train',
    f'{OUTPUT_BASE_DIR}/labels/val'
]:
    os.makedirs(dir_path)

# словарь классов
classes = {
    'title': 0,
    'paragraph': 1,
    'table': 2,
    'picture': 3,
    'table_signature': 4,
    'picture_signature': 5,
    'numbered_list': 6,
    'marked_list': 7,
    'header': 8,
    'footer': 9,
    'footnote': 10,
    'formula': 11
}

def get_matching_files():
    """
    Получаем список файлов, у которых есть соответствующие пары json и png
    """
    image_files = {f.stem for f in Path(SOURCE_IMAGES_DIR).glob('*.png')}
    json_files = {f.stem for f in Path(SOURCE_JSONS_DIR).glob('*.json')}
    
    # находим пересечение множеств - файлы, у которых есть обе версии
    matching_files = sorted(list(image_files.intersection(json_files)))
    print(f"Total images found: {len(image_files)}")
    print(f"Total JSONs found: {len(json_files)}")
    print(f"Matching pairs found: {len(matching_files)}")
    return matching_files

def process_files(file_list, train_ratio=0.8):
    """
    Обрабатываем файлы, случайно распределяя их между train и val
    """
    # перемешиваем список файлов
    random.seed(42)  # для воспроизводимости
    random.shuffle(file_list)
    
    # определяем точку разделения
    split_point = int(len(file_list) * train_ratio)
    train_files = file_list[:split_point]
    val_files = file_list[split_point:]
    
    print(f"\nSplitting {len(file_list)} files:")
    print(f"Training files: {len(train_files)}")
    print(f"Validation files: {len(val_files)}")
    
    processed_train = 0
    processed_val = 0
    
    # обрабатываем тренировочные и валидационные файлы
    for is_train, files in [(True, train_files), (False, val_files)]:
        subset = 'train' if is_train else 'val'
        
        for filename in files:
            try:
                # копируем изображение
                src_img = os.path.join(SOURCE_IMAGES_DIR, f'{filename}.png')
                dst_img = os.path.join(OUTPUT_BASE_DIR, 'images', subset, f'{filename}.png')
                shutil.copy2(src_img, dst_img)
                
                # обрабатываем JSON и создаем YOLO-формат
                src_json = os.path.join(SOURCE_JSONS_DIR, f'{filename}.json')
                dst_txt = os.path.join(OUTPUT_BASE_DIR, 'labels', subset, f'{filename}.txt')
                
                with open(src_json) as f:
                    templates = json.loads(f.read())
                    
                with open(dst_txt, "w") as g:
                    for name in classes.keys():
                        if name in templates and len(templates[name]) > 0:
                            for obj in templates[name]:
                                g.write(f"{classes[name]} {((obj[2] + obj[0]) / 2) / templates['image_width']} "
                                      f"{((obj[3] + obj[1]) / 2) / templates['image_height']} "
                                      f"{(obj[2] - obj[0]) / templates['image_width']} "
                                      f"{(obj[3] - obj[1]) / templates['image_height']}\n")
                
                if is_train:
                    processed_train += 1
                else:
                    processed_val += 1
                    
            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")
                continue
    
    return processed_train, processed_val

def create_yaml_config():
    """
    Создаем конфигурационный YAML файл
    """
    data = {
        "path": OUTPUT_BASE_DIR,
        "train": 'images/train',
        "val": 'images/val',
        "names": {v: k for k, v in classes.items()}  
    }
    
    config_path = '/kaggle/working/config.yaml'
    with open(config_path, "w") as f:
        yaml.dump(data, f)
    return config_path

def main():
    # получаем список файлов
    matching_files = get_matching_files()
    
    # обрабатываем файлы
    processed_train, processed_val = process_files(matching_files)
    
    # создаем конфигурацию
    config_path = create_yaml_config()
    
    # выводим финальную статистику
    print(f"\nFinal statistics:")
    print(f"Successfully processed training images: {processed_train}")
    print(f"Successfully processed validation images: {processed_val}")
    print(f"Total processed: {processed_train + processed_val}")
    
    return config_path

# запускаем подготовку данных
config_path = main()

# Обучение модели

Ещё кое-какие установки для исключения ошибок:

In [ ]:
!pip install -U ipywidgets -q
!pip uninstall ray[tune] -y -q

In [ ]:
!pip install doclayout_yolo -q

## Обучение модели на GPU

### Этап 1: Обучение с замороженным первым слоем

Модель, которую я буду использовать для дообучения - предобученная.
Поэтому я заморожу первый слой модели, а также буду использовать низкий lr и оптимизатор AdamW для дообучения на моих данных.

In [ ]:
import os
import torch
from doclayout_yolo import YOLOv10

# очистка памяти и настройка CUDA
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# загрузка предобученной модели
print("Загрузка предобученной модели...")
model = YOLOv10("/kaggle/input/superpretrained_yolov10/pytorch/default/1/doclayout_yolo_docstructbench_imgsz1024.pt")

results_stage1 = model.train(
    data=str(config_path),
    epochs=10,
    imgsz=800,  
    batch=4,
    workers=4,
    project='DocLayout YOLO',
    name="experiment_stage1",
    exist_ok=True,
    amp=False,
    cache=False,
    pretrained=True,
    resume=False,
    verbose=True,
    freeze=[0], 
    optimizer="AdamW",  
    lr0=0.001,
    lrf=0.01,
    momentum=0.935,
    weight_decay=0.001,  
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    plots=True,
    save=True,
    save_period=2,
    dropout=0.1, 
    multi_scale=False,
    augment=True,  
    degrees=2.0,  
    hsv_h=0.1,    
    hsv_s=0.1, 
    hsv_v=0.1, 
    mosaic=0.0,   
    flipud=0.0,   
    fliplr=0.0    
)
print("Этап 1 завершен!")

### Этап 2: Разморозка всех слоёв и тонкая настройка

На этом этапе я разморожу вообще все слои и буду с очень низким lr дообучать модель, полученную на предыдущем этапе.

In [ ]:
# print("Этап 2: Разморозка всех слоёв и тонкая настройка всей модели...")
# results_stage2 = model.train(
#     data=str(config_path),
#     epochs=5,
#     imgsz=800, 
#     batch=4,
#     workers=4,
#     project='DocLayout YOLO',
#     name="experiment_stage2",
#     exist_ok=True,
#     amp=False,
#     cache=False,
#     pretrained=True,
#     resume=True, 
#     verbose=True,
#     freeze=[],  
#     optimizer="AdamW",
#     lr0=0.0001,  
#     lrf=0.01,
#     momentum=0.935,
#     weight_decay=0.001,
#     warmup_epochs=1.0,
#     warmup_momentum=0.8,
#     warmup_bias_lr=0.05,
#     box=7.5,
#     cls=0.5,
#     dfl=1.5,
#     plots=True,
#     save=True,
#     save_period=1,
#     dropout=0.1,  
#     multi_scale=False,
#     augment=True,  
#     degrees=2.0, 
#     hsv_h=0.1, 
#     hsv_s=0.1, 
#     hsv_v=0.1, 
#     mosaic=0.0,   
#     flipud=0.0,  
#     fliplr=0.0    
# )
# print("Этап 2 завершен!")


# Результаты

Метрики находятся в одной папке с ноутбуком.